<a href="https://colab.research.google.com/github/biamouras/alura_pyspark_challenge/blob/main/Notebooks/02_tratamento_dados_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insightplaces - Tratamento dos dados para ML

A Insightplaces, plataforma fantasia de imóveis da Alura, está com dificuldades para definir os valores do imóveis e gostaria de fazer de modo automatizado. Ainda, a empresa percebeu que o recomendador atual não está sendo efetivo para gerar novos cliques entre os usuários.

Nosso projeto será trabalhar com os dados disponibilizados de imóveis no Rio de Janeiro para ajudar a empresa a lidar com estes dois problemas usando regressão para a definição dos valores e a análise de agrupamentos através de Machine Learning. 

Este segundo projeto tem o foco de preparar os dados para os modelos de previsão a partir de dados já tratados.

## Carregando os dados

In [1]:
from os import environ, listdir

In [2]:
# instalar as dependências para utilizar Spark no Colab
instala = True

# verifica se já está instalado 
for f in listdir('/content'):
  if f.find('spark') != -1:
    instala = False
    break 

if instala:
  !apt-get update -qq
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null

  !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
  !tar xf spark-3.1.2-bin-hadoop2.7.tgz
  !pip install -q findspark

environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

In [3]:
# iniciando o pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [4]:
# monta a conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# biblioteca para abrir o arquivo compactado
import zipfile

In [6]:
# caminho do arquivo 
name = 'semana-2'
dir_dados = '/content/drive/MyDrive/Colab Notebooks/alura_challenge_insightplaces/dados/' + name + '/'

# descompressão do arquivo
zipfile.ZipFile(dir_dados + name + '.zip', 'r').extractall(dir_dados)
# identificação do arquivo descomprimido
files = listdir(dir_dados)
files 

['semana-2.zip', 'dataset_transformado_parquet']

In [7]:
# o zip contém uma pasta
dir_dados_originais = dir_dados + files[1]
# verifica os arquivos dentro da pasta comprimida
files = listdir(dir_dados_originais)
files

['._SUCCESS.crc',
 '_SUCCESS',
 '.part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet.crc',
 'part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet']

In [48]:
# leitura do parquet
parquet_file = [f for f in files if f.endswith('.parquet')][0]
dados = spark.read.parquet(dir_dados_originais + '/' + parquet_file)
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## Correção do tipo de  variável

Analisando os tipos de dados das colunas, vemos que features que deveriam ser numéricas estão consideradas como `string`. 

In [49]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [26]:
import pyspark.sql.types as types

In [50]:
# convertendo area_total, area_util, condominio, iptu e valor
double_cols = ['area_total', 'area_util', 'condominio', 'iptu', 'valor']

for col in double_cols:
  dados = dados\
          .withColumn(col, dados[col].cast(types.DoubleType()))

dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



Outras variáveis, como `andar`, `banheiros`, `quartos`, `suites` e `vagas`, estão como `long` e podem ocupar muito espaço na memória sem necessidade. Portanto, vamos convertê-las para ShortType.

In [51]:
# colunas para ShortType
short_cols = ['andar', 'banheiros', 'quartos', 'suites', 'vaga']

# verificando limites das variáveis
dados\
  .select(short_cols)\
  .summary()\
  .show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|             andar|         banheiros|           quartos|            suites|              vaga|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|             66562|             66562|             66562|             61008|             63545|
|   mean|2.3374598119046905| 2.454583696403353|2.6121811243652533|1.2091528979805928|1.4069084900464237|
| stddev|  14.6963496227043|1.3582533594407473|0.9083944981999731|1.0567921717756774|1.3413027342394985|
|    min|                 0|                 1|                 0|                 0|                 0|
|    25%|                 0|                 2|                 2|                 1|                 1|
|    50%|                 0|                 2|                 3|                 1|                 1|
|    75%|                 3|                 3|        

Como os valores máximos não ultrapassam o limite de ShortType (entre -32868 a 32767), podemos realizar a conversão.

In [52]:
for col in short_cols:
  dados = dados\
                .withColumn(col, dados[col].cast(types.ShortType()))

In [53]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: short (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: short (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: short (nullable = true)
 |-- suites: short (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: short (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



## Tratamento de valores Nulos

Antes de realizar o tratamento dos valores nulos, vamos trata a coluna `caracteristicas` que contém algumas listas vazias.

In [33]:
import pyspark.sql.functions as f

In [62]:
# como intermediario, concatena a lista dentro de características, 
# verifica o tamanho do string
# se for igual a 0, a lista está vazia, então retorna null,
# senão, retorna o valor original
dados = dados\
      .withColumn('caracteristicas', f.when(f.length(f.concat_ws(', ', 'caracteristicas')) == 0, None).otherwise(dados.caracteristicas))

dados.select('caracteristicas').show(10)

+--------------------+
|     caracteristicas|
+--------------------+
|[Churrasqueira, A...|
|                null|
|                null|
|                null|
|                null|
|[Condomínio fecha...|
|[Churrasqueira, C...|
|[Churrasqueira, P...|
|[Churrasqueira, E...|
|   [Salão de festas]|
+--------------------+
only showing top 10 rows



Analisando a quantidade de observações válidas (66.562), vemos que as features `area_total`, `suites`, `vaga`, `condominio`, e `iptu` têm menos registros, indicando a presença de observações nulas.

In [64]:
# seleciona variáveis numéricas
var_numericas = [var for var, types in dados.dtypes if 'string' not in types]

# identifica a quantidade de NA em cada coluna
dados\
  .select(var_numericas)\
  .summary()\
  .show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|             andar|       area_total|         area_util|         banheiros|           quartos|            suites|              vaga|        condominio|              iptu|             valor|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             66562|            57368|             66562|             66562|             66562|             61008|             63545|             64191|             59363|             66562|
|   mean|2.3374598119046905|938.1081264816622|116.73728253357771| 2.454583696403353|2.6121811243652533|1.2091528979805928|1.4069084900464237|4822.7975728684705|5447.7366036083085|1294433.2853129413|
| std

De modo genérico, vamos preencher os valores nulos pelas médias de cada variável em relação aos bairros.

In [78]:
from pyspark.sql import Window

# variáveis que precisam de preenchimento
var_na = ['area_total', 'suites', 'vaga', 'condominio', 'iptu']

# calcula as médias para cada feature (criar um dicionário)
w = Window.partitionBy(dados.bairro)

df = dados

for col in var_na:
  df = df\
  .withColumn(col, 
              f.when(f.isnan(col) | f.isnull(col), 
                          f.avg(col).over(w))\
              .otherwise(dados[col]))

df.select(var_na + ['bairro']).show(10)

+-----------------+-------------------+----+----------+------------------+------+
|       area_total|             suites|vaga|condominio|              iptu|bairro|
+-----------------+-------------------+----+----------+------------------+------+
|             77.0|                0.0| 1.0|     350.0|              76.0|Cocotá|
|62.53846153846154|                0.0| 1.0|     310.0|19367.384615384617|Cocotá|
|             46.0|                0.0| 1.0|     370.0|              25.0|Cocotá|
|             55.0|0.07692307692307693| 1.0|     315.0|               0.0|Cocotá|
|             56.0|                0.0| 1.0|     420.0|               0.0|Cocotá|
|62.53846153846154|0.07692307692307693| 1.0|     210.0|             650.0|Cocotá|
|            112.0|0.07692307692307693| 1.0|     400.0|               0.0|Cocotá|
|             60.0|                0.0| 1.0|     300.0|               0.0|Cocotá|
|             44.0|                0.0| 2.0|     300.0|               0.0|Cocotá|
|             46

In [79]:
# verifica a quantidade de NA em cada coluna tratada
df\
  .select(var_na)\
  .summary()\
  .show()

+-------+-----------------+------------------+------------------+-----------------+------------------+
|summary|       area_total|            suites|              vaga|       condominio|              iptu|
+-------+-----------------+------------------+------------------+-----------------+------------------+
|  count|            66562|             66558|             66559|            66560|             66556|
|   mean|966.3603330599384|1.1766074024133777|1.3883332557344061|4795.622909567858| 5393.876817238255|
| stddev|  78149.954199951|1.0254394312575272|1.3158729395552726|104147.6873313818|171136.61673109786|
|    min|              0.0|               0.0|               0.0|              0.0|               0.0|
|    25%|             70.0|               1.0|               1.0|            475.0|              80.0|
|    50%|             97.0|               1.0|               1.0|            900.0|             482.0|
|    75%|            154.0|               1.0|               2.0|        

## Seleção de features

Devido a alguns filtros realizados nos dados brutos em relação as colunas: `tipo_uso`, `tipo_unidade` e `tipo_anuncio`, podemos desconsiderá-las para os modelos.

Antes, vamos verificar se de fatos os valores para estas colunas são únicos.

In [83]:
# verificação da variabilidade
df.createOrReplaceTempView('dfView')
spark\
  .sql("""
        SELECT DISTINCT tipo_uso, tipo_unidade, tipo_anuncio, tipo
        FROM dfView
  """)\
  .show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio| tipo|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|Venda|
+-----------+------------+------------+-----+



In [84]:
df = df.drop('tipo_uso', 'tipo_unidade', 'tipo_anuncio', 'tipo')
df.show(5)

+--------------------+-----+-----------------+---------+---------+--------------------+-------+-------------------+----+------+----------+----------+------------------+--------+
|                  id|andar|       area_total|area_util|banheiros|     caracteristicas|quartos|             suites|vaga|bairro|      zona|condominio|              iptu|   valor|
+--------------------+-----+-----------------+---------+---------+--------------------+-------+-------------------+----+------+----------+----------+------------------+--------+
|de2485bb-8f82-4e7...|    0|             77.0|     77.0|        2|[Elevador, Salão ...|      2|                0.0| 1.0|Cocotá|Zona Norte|     350.0|              76.0|345000.0|
|7aea937e-6493-4c9...|    3|62.53846153846154|     45.0|        1|                null|      1|                0.0| 1.0|Cocotá|Zona Norte|     310.0|19367.384615384617|180000.0|
|f8738a5e-064d-421...|    0|             46.0|     46.0|        1|[Churrasqueira, S...|      2|               

Temos ainda as variáveis `area_total` e `area_util`. Considerando que houve a necessidade de tratamento da `area_total` pela falta de informação, e que há grande variabilidade entre os valores desta variável, também vamos descartá-la.

In [87]:
df\
  .select(['area_total', 'area_util'])\
  .summary()\
  .show()

+-------+-----------------+------------------+
|summary|       area_total|         area_util|
+-------+-----------------+------------------+
|  count|            66562|             66562|
|   mean|966.3603330599384|116.73728253357771|
| stddev|  78149.954199951| 89.09094364150404|
|    min|              0.0|              10.0|
|    25%|             70.0|              67.0|
|    50%|             97.0|              90.0|
|    75%|            154.0|             138.0|
|    max|      1.5022793E7|            3456.0|
+-------+-----------------+------------------+



In [88]:
df = df.drop('area_total')
df.show(5)

+--------------------+-----+---------+---------+--------------------+-------+-------------------+----+------+----------+----------+------------------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|             suites|vaga|bairro|      zona|condominio|              iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+-------------------+----+------+----------+----------+------------------+--------+
|de2485bb-8f82-4e7...|    0|     77.0|        2|[Elevador, Salão ...|      2|                0.0| 1.0|Cocotá|Zona Norte|     350.0|              76.0|345000.0|
|7aea937e-6493-4c9...|    3|     45.0|        1|                null|      1|                0.0| 1.0|Cocotá|Zona Norte|     310.0|19367.384615384617|180000.0|
|f8738a5e-064d-421...|    0|     46.0|        1|[Churrasqueira, S...|      2|                0.0| 1.0|Cocotá|Zona Norte|     370.0|              25.0|230000.0|
|62b14fe0-16a9-405...|    0|     55.0|  